In [3]:
from PIL import Image
import os.path
from os import path
import math
from Crypto.Cipher import AES
from Crypto.Hash import SHA256
from Crypto import Random
import base64
import getpass
import sys

DEBUG = False
headerText = "M6nMjy5THr2J"

def encrypt(key, source, encode=True):
    key = SHA256.new(key).digest()
    IV = Random.new().read(AES.block_size)
    encryptor = AES.new(key, AES.MODE_CBC, IV)
    padding = AES.block_size - len(source) % AES.block_size
    source += bytes([padding]) * padding
    data = IV + encryptor.encrypt(source)
    return base64.b64encode(data).decode() if encode else data

def decrypt(key, source, decode=True):
    if decode:
        source = base64.b64decode(source.encode())
    key = SHA256.new(key).digest()
    IV = source[:AES.block_size]
    decryptor = AES.new(key, AES.MODE_CBC, IV)
    data = decryptor.decrypt(source[AES.block_size:])
    padding = data[-1]
    if data[-padding:] != bytes([padding]) * padding:
        raise ValueError("Invalid padding...")
    return data[:-padding]


def convertToRGB(img):
    try:
        rgba_image = img
        rgba_image.load()
        background = Image.new("RGB", rgba_image.size, (255, 255, 255))
        background.paste(rgba_image, mask=rgba_image.split()[3])
        print("Converted image to RGB")
        return background
    except Exception as e:
        print("Couldn't convert image to RGB - %s" % e)

def getPixelCount(img):
    width, height = Image.open(img).size
    return width * height

def encodeImage(image, message, filename):
    print("Encoding image...")
    try:
        width, height = image.size
        pix = image.getdata()

        current_pixel = 0
        tmp = 0
        x = 0
        y = 0
        for ch in message:
            binary_value = format(ord(ch), '08b')

            # For each character, get 3 pixels at a time
            p1 = pix[current_pixel]
            p2 = pix[current_pixel+1]
            p3 = pix[current_pixel+2]

            three_pixels = [val for val in p1 + p2 + p3]

            for i in range(0, 8):
                current_bit = binary_value[i]

                # 0 - Even
                # 1 - Odd
                if current_bit == '0':
                    if three_pixels[i] % 2 != 0:
                        three_pixels[i] = three_pixels[i] - 1 if three_pixels[i] == 255 else three_pixels[i] + 1
                elif current_bit == '1':
                    if three_pixels[i] % 2 == 0:
                        three_pixels[i] = three_pixels[i] - 1 if three_pixels[i] == 255 else three_pixels[i] + 1

            current_pixel += 3
            tmp += 1

            # Set 9th value
            if tmp == len(message):
                # Make as 1 (odd) - stop reading
                if three_pixels[-1] % 2 == 0:
                    three_pixels[-1] = three_pixels[-1] - 1 if three_pixels[-1] == 255 else three_pixels[-1] + 1
            else:
                # Make as 0 (even) - continue reading
                if three_pixels[-1] % 2 != 0:
                    three_pixels[-1] = three_pixels[-1] - 1 if three_pixels[-1] == 255 else three_pixels[-1] + 1

            if DEBUG:
                print("Character: ", ch)
                print("Binary: ", binary_value)
                print("Three pixels before mod: ", three_pixels)
                print("Three pixels after mod: ", three_pixels)

            three_pixels = tuple(three_pixels)

            st = 0
            end = 3

            for i in range(0, 3):
                if DEBUG:
                    print("Putting pixel at ", (x, y), " to ", three_pixels[st:end])

                image.putpixel((x, y), three_pixels[st:end])
                st += 3
                end += 3

                if x == width - 1:
                    x = 0
                    y += 1
                else:
                    x += 1

        downloads_folder = os.path.join(os.path.expanduser("~"), "Downloads")
        encoded_filename = os.path.join(downloads_folder, filename.split('.')[0] + "-enc.png")
        image.save(encoded_filename)
        print("\n")
        print("Original File: %s" % filename)
        print("Image encoded and saved as %s" % encoded_filename)

    except Exception as e:
        print("An error occurred - %s" % e)
        sys.exit(0)

def decodeImage(image):
    print("Decoding image...")
    try:
        pix = image.getdata()
        current_pixel = 0
        decoded = ""
        while True:
            # Get 3 pixels each time
            binary_value = ""
            p1 = pix[current_pixel]
            p2 = pix[current_pixel+1]
            p3 = pix[current_pixel+2]
            three_pixels = [val for val in p1 + p2 + p3]

            for i in range(0, 8):
                if three_pixels[i] % 2 == 0:
                    # add 0
                    binary_value += "0"
                elif three_pixels[i] % 2 != 0:
                    # add 1
                    binary_value += "1"

            # Convert binary value to ascii and add to string
            binary_value.strip()
            ascii_value = int(binary_value, 2)
            decoded += chr(ascii_value)
            current_pixel += 3

            if DEBUG:
                print("Binary: ", binary_value)
                print("Ascii: ", ascii_value)
                print("Character: ", chr(ascii_value))

            if three_pixels[-1] % 2 != 0:
                # stop reading
                break

        return decoded
    except Exception as e:
        print("An error occurred - %s" % e)
        sys.exit()

def main():
    print("Choose one: ")
    op = int(input("1. Encode\n2. Decode\n>>"))

    if op == 1:
        print("Image path (with extension): ")
        img = input(">>")
        
        # Remove the quotes from the input path
        img = img.strip('"')
        
        if not path.exists(img):
            raise Exception("Image not found!")

        print("Message to be hidden: ")
        message = input(">>")
        message = headerText + message
        if (len(message) + len(headerText)) * 3 > getPixelCount(img):
            raise Exception("Given message is too long to be encoded in the image.")

        password = ""
        while True:
            print("Password to encrypt (leave empty if you want no password): ")
            password = getpass.getpass(">>")
            if password == "":
                break
            print("Re-enter Password: ")
            confirm_password = getpass.getpass(">>")
            if password != confirm_password:
                print("Passwords don't match, try again")
            else:
                break

        cipher = ""
        if password != "":
            cipher = encrypt(key=password.encode(), source=message.encode())
            cipher = headerText + cipher
        else:
            cipher = message

        if DEBUG:
            print("Encrypted: ", cipher)

        image = Image.open(img)
        if image.mode != 'RGB':
            image = convertToRGB(image)
        newimg = image.copy()
        
        # Get the filename from the path
        filename = os.path.basename(img)
        encodeImage(image=newimg, message=cipher, filename=filename)

    elif op == 2:
        print("Image path (with extension): ")
        img = input(">>")
        
        # Remove the quotes from the input path
        img = img.strip('"')
        
        if not path.exists(img):
            raise Exception("Image not found!")

        print("Enter password (leave empty if no password): ")
        password = getpass.getpass(">>")

        image = Image.open(img)
        cipher = decodeImage(image)
        header = cipher[:len(headerText)]

        if header.strip() != headerText:
            print("Invalid data!")
            sys.exit(0)

        if DEBUG:
            print("Decoded text: %s" % cipher)

        decrypted = ""

        if password != "":
            cipher = cipher[len(headerText):]
            try:
                decrypted = decrypt(key=password.encode(), source=cipher)
            except Exception as e:
                print("Wrong password!")
                sys.exit(0)
        else:
            decrypted = cipher

        header = decrypted.decode()[:len(headerText)]

        if header != headerText:
            print("Wrong password!")
            sys.exit(0)

        decrypted = decrypted[len(headerText):]

        print("Decoded Text: \t%s" % decrypted)

if __name__ == "__main__":
    print("Hide Simple Text Hiding in Images")
    main()


Hide Simple Text Hiding in Images
Choose one: 
1. Encode
2. Decode
>>2
Image path (with extension): 
>>"C:\Users\ansar\Downloads\pexels-pixabay-56866-enc.png"
Enter password (leave empty if no password): 
>>········
Decoding image...
Decoded Text: 	b'hiding a message'
